# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,brandfort,-28.7001,26.4597,56.61,36.0,2.0,6.93,ZA,1717089706
1,1,idri,27.5000,13.2667,98.04,9.0,0.0,8.63,LY,1717089707
2,2,san andros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
3,3,tiksi,71.6872,128.8694,36.81,88.0,55.0,2.57,RU,1717089474
4,4,thinadhoo,0.5333,72.9333,84.31,81.0,83.0,19.46,MV,1717089709


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     599 non-null    int64  
 1   City        599 non-null    object 
 2   Lat         565 non-null    float64
 3   Long        565 non-null    float64
 4   Max Temp    565 non-null    float64
 5   Humidity    565 non-null    float64
 6   Cloudiness  565 non-null    float64
 7   Wind Speed  565 non-null    float64
 8   Country     560 non-null    object 
 9   Date        599 non-null    object 
dtypes: float64(6), int64(1), object(3)
memory usage: 46.9+ KB


In [4]:
city_data_df.dropna(inplace=True)
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 560 entries, 0 to 598
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     560 non-null    int64  
 1   City        560 non-null    object 
 2   Lat         560 non-null    float64
 3   Long        560 non-null    float64
 4   Max Temp    560 non-null    float64
 5   Humidity    560 non-null    float64
 6   Cloudiness  560 non-null    float64
 7   Wind Speed  560 non-null    float64
 8   Country     560 non-null    object 
 9   Date        560 non-null    object 
dtypes: float64(6), int64(1), object(3)
memory usage: 48.1+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Long",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    s = "Humidity",
    color = "Humidity",
    title = "Map of Cities by Humidity"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Long,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
ideal_df = city_data_df.copy()
ideal_df = ideal_df.loc[(ideal_df["Max Temp"] < 80) & (ideal_df["Max Temp"] > 60)].reset_index(drop=True)
ideal_df = ideal_df.loc[(ideal_df["Wind Speed"] < 4.5)].reset_index(drop=True)
ideal_df = ideal_df.loc[(ideal_df["Cloudiness"] < 20)].reset_index(drop=True)

# Display sample data
ideal_df

,City_ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,31,rio grande,-32.0350,-52.0986,64.51,68.0,8.0,1.16,BR,1717089725
1,56,tilcara,-23.5776,-65.3509,61.52,10.0,13.0,1.30,AR,1717089741
2,135,ibarreta,-25.2144,-59.8585,72.70,40.0,0.0,3.04,AR,1717089790
3,214,cidreira,-30.1811,-50.2056,63.75,62.0,6.0,3.36,BR,1717089674
4,233,mananjary,-21.2167,48.3333,70.36,72.0,8.0,2.73,MG,1717089489
5,291,amanalco de becerra,19.3167,-100.0167,69.22,29.0,1.0,2.62,MX,1717089889
6,322,mutuali,-14.8706,37.0044,67.19,47.0,0.0,2.44,MZ,1717089908
7,445,arkhangel'sk,64.5401,40.5433,64.51,42.0,0.0,4.47,RU,1717089987
8,464,bonito,-21.1211,-56.4819,77.07,32.0,0.0,3.71,BR,1717089999
9,474,big bend,-26.8167,31.9333,62.69,88.0,0.0,0.00,SZ,1717090006


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
cols = ["City", "Country", "Lat", "Long", "Humidity"]
hotel_df = ideal_df[cols].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = pd.Series(dtype='object')

# Display sample data
hotel_df

,City,Country,Lat,Long,Humidity,Hotel Name
0,rio grande,BR,-32.0350,-52.0986,68.0,NaN
1,tilcara,AR,-23.5776,-65.3509,10.0,NaN
2,ibarreta,AR,-25.2144,-59.8585,40.0,NaN
3,cidreira,BR,-30.1811,-50.2056,62.0,NaN
4,mananjary,MG,-21.2167,48.3333,72.0,NaN
5,amanalco de becerra,MX,19.3167,-100.0167,29.0,NaN
6,mutuali,MZ,-14.8706,37.0044,47.0,NaN
7,arkhangel'sk,RU,64.5401,40.5433,42.0,NaN
8,bonito,BR,-21.1211,-56.4819,32.0,NaN
9,big bend,SZ,-26.8167,31.9333,88.0,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1

params = {"categories":categories,
        "limit":limit,
        "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Long"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    status_code = response.status_code
    
    if status_code == 200:
        name_address = response.json()
    else:
        name_address = {} # failed request
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rio grande - nearest hotel: Hotel Vila Moura Executivo
tilcara - nearest hotel: Casa Colorada - Hotel de Montaña
ibarreta - nearest hotel: Hotel Plaza
cidreira - nearest hotel: Hotel Castelo
mananjary - nearest hotel: Ideal Hotel Annexe
amanalco de becerra - nearest hotel: No hotel found
mutuali - nearest hotel: No hotel found
arkhangel'sk - nearest hotel: Серафима
bonito - nearest hotel: Hotel Pousada Arauna
big bend - nearest hotel: No hotel found
mahina - nearest hotel: Motu Martin
mumbwa - nearest hotel: La Hacienda


,City,Country,Lat,Long,Humidity,Hotel Name
0,rio grande,BR,-32.0350,-52.0986,68.0,Hotel Vila Moura Executivo
1,tilcara,AR,-23.5776,-65.3509,10.0,Casa Colorada - Hotel de Montaña
2,ibarreta,AR,-25.2144,-59.8585,40.0,Hotel Plaza
3,cidreira,BR,-30.1811,-50.2056,62.0,Hotel Castelo
4,mananjary,MG,-21.2167,48.3333,72.0,Ideal Hotel Annexe
5,amanalco de becerra,MX,19.3167,-100.0167,29.0,No hotel found
6,mutuali,MZ,-14.8706,37.0044,47.0,No hotel found
7,arkhangel'sk,RU,64.5401,40.5433,42.0,Серафима
8,bonito,BR,-21.1211,-56.4819,32.0,Hotel Pousada Arauna
9,big bend,SZ,-26.8167,31.9333,88.0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Long",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    s = "Humidity",
    color = "Humidity",
    hover_cols = "all",
    title = "Map of Hotels in Select Cities by Humidity"
)
    
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Long,Lat]   (Humidity,index,City,Country,Hotel Name)